<a href="https://colab.research.google.com/github/lalithakre/NLP-Sentiment-Analysis/blob/main/(NLP)Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading word embedding


Paragraph ko sentences
sentences ko words
words ko base word mai like...given ko give mai
stop words remove karna..like comma 
padding.. means joh

In [ ]:
!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip

--2023-01-11 08:51:41--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.04MB/s    in 2m 40s  

2023-01-11 08:54:21 (5.12 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [ ]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


creating data frame


In [ ]:
import pandas as pd
data=[['the restaurant is awesome',1],
      ['the food is so oily and spicy',0],
      ['food is ok',1]
      ]
df=pd.DataFrame(data,columns=['review','sentiment'])
df

,review,sentiment
0,the restaurant is awesome,1
1,the food is so oily and spicy,0
2,food is ok,1


In [ ]:
def add_to_dict(filename,d):
  import numpy as np 
  with open(filename,'r') as f:
    for line in f.readlines(): #ek file ki ek ek line ko read karta hai... 
      l=line.split(' ')
      d[l[0]]=np.array(l[1:],dtype='float')
      

In [ ]:
dics={}
add_to_dict('glove.6B.50d.txt', dics)


In [ ]:
dics['given']

array([ 0.43768  ,  0.5543   , -0.27591  , -0.36805  ,  0.82612  ,
        0.14132  , -0.0063327,  0.17197  , -0.0042713,  0.14954  ,
        0.30217  ,  0.43632  ,  0.053702 , -0.59632  ,  0.64761  ,
       -0.024799 , -0.016603 , -0.40788  , -0.24985  , -0.58336  ,
       -0.13173  , -0.18747  ,  0.26913  , -0.25625  ,  0.021073 ,
       -1.6454   , -0.53461  , -0.12627  , -0.09779  ,  0.2995   ,
        3.3942   ,  0.19733  , -0.31749  , -0.62559  ,  0.52945  ,
        0.067728 ,  0.49578  ,  0.27542  , -0.35416  , -0.37977  ,
        0.20921  ,  0.34261  , -0.32328  ,  0.53135  , -0.53123  ,
       -0.39414  , -0.12909  ,  0.54714  ,  0.071181 ,  0.015065 ])

In [ ]:
# sentences=['this is my favourite restaurant',
#           'i like to eat burger in this restaurant'
#           'food is so spicy not eatable',
#            'food is ok']
# sentiment=['positive','positive','negetive','nutral'] # 3 classes

In [ ]:
import nltk #natural language tool kit
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# words=[]
# for i in sentences:
#   p=word_tokenize(i)
#   print(p)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize # sentence ko words mai tood denge...sent_tokenuze..para ko sent mai...
Lemmatizer=WordNetLemmatizer()


In [ ]:
def message_to_tokenslist(s):
  tokens=word_tokenize(s)
  lower_case = [t.lower() for t in tokens] #lower case mai laaya..
  lemm=[Lemmatizer.lemmatize(t) for t in lower_case] # base form mai laya..
  usefultokens=[t for t in lemm if t in dics]  # useful tokens joh dics mai hai.
  return usefultokens


In [ ]:
# message_to_tokenslist(' I Am so happy at  sistec',dics)

In [ ]:
import numpy as np

def message_to_vector(s,dics):
   usefultokens=message_to_tokenslist(s)
   vector=[]
   for token in usefultokens:
     if token not in dics:
       continue
     else:
       q=dics[token]
       vector.append(q)
   return np.array(vector,dtype='float')



In [ ]:
a=message_to_vector('hello how are you',dics)
a.shape

(4, 50)

In [ ]:
def df_to_x_y(df,dics):
  y=df['sentiment'].to_numpy().astype('int') 
  x=[]
  for message in df['review']:
    message_in_metrix=message_to_vector(message,dics)
    x.append(message_in_metrix)
  return x,y 


In [ ]:
from  copy import deepcopy   # ek ka change doosre mai nhi aayega..bcoz of deepcopy
import numpy as np
def pad_x(x,length):
  x_copy=[]
  for i in x:
    difference=length-i.shape[0]
    pad=np.zeros(shape=(difference,50))
    x_copy.append(np.concatenate([i,pad]))

  return  np.array(x_copy)

In [ ]:
x,y=df_to_x_y(df,dics)

In [ ]:
xarr=pad_x(x,7)
xarr.shape

(3, 7, 50)

In [ ]:
from keras.models import Sequential

model=Sequential()
from keras.layers import Dense,LSTM,Dropout,Input
model.add(Input(shape=(7,50)))
model.add(LSTM(64,activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,activation='relu',return_sequences=False))
model.add(Dropout(0.2)) #kitne nurons ko stop karna hai..
model.add(Dense(1,activation='sigmoid'))# binary classification
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xarr,y,epochs=10)


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.6928 - accuracy: 0.6667
Epoch 2/10
1/1 [==============================] - 0s 20ms/step - loss: 0.6909 - accuracy: 0.6667
Epoch 3/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6959 - accuracy: 0.6667
Epoch 4/10
1/1 [==============================] - 0s 22ms/step - loss: 0.6620 - accuracy: 0.6667
Epoch 5/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6540 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 24ms/step - loss: 0.6533 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 23ms/step - loss: 0.6529 - accuracy: 0.6667
Epoch 8/10
1/1 [==============================] - 0s 18ms/step - loss: 0.6493 - accuracy: 0.6667
Epoch 9/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6291 - accuracy: 0.6667
Epoch 10/10
1/1 [==============================] - 0s 19ms/step - loss: 0.6128 - accuracy: 1.0000


In [ ]:
o=model.predict(xarr)
o>0.5

1/1 [==============================] - 0s 285ms/step


array([[ True],
       [False],
       [ True]])